In [1]:
from flurs.datasets import csv_loader
from flurs.recommender import BRISMFRecommender, MFRecommender
from flurs.forgetting import ForgetUnpopularItems, NoForgetting, MappedUserFactorFading, UserFactorFading, SDUserFactorFading
from flurs.evaluator import Evaluator

import logging
import os
import sys

ML_PATH = 'D:\\recsys\\datasets\\MovieLens\\ml-1m\\ml-1m-gte.csv'
EXP_NAME = "M2"
RECALL_AT = 10

program = os.path.basename(ML_PATH)
logger = logging.getLogger(program)

logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s')
logging.root.setLevel(level=logging.INFO)
logger.info('running %s' % ' '.join(sys.argv))

logging.info('converting data into FluRS input object')
data = csv_loader(ML_PATH)

2019-03-25 23:19:42,478 : INFO : running D:\Anaconda3\envs\recsys\lib\site-packages\ipykernel_launcher.py -f C:\Users\Eduardo Ferreira\AppData\Roaming\jupyter\runtime\kernel-25d0cedd-48f9-42b9-88d1-1d1b96c52418.json
2019-03-25 23:19:42,482 : INFO : converting data into FluRS input object


In [2]:
class Experimenter:
    def __init__(self):
        self._recommenders = []
        self.forgettings = []
    def append(self, recommender_list, forgetting_list, parameters_list):
        for f in forgetting_list:
            self.forgettings.append(f.__name__)
        for Recommender in recommender_list:
            for p in parameters_list:
                for Forgetting in forgetting_list:
                    self._recommenders.append(Recommender(forgetting = Forgetting(alpha=p), learn_rate=.002, k=60))
                    
    def __getitem__(self, position):
        return self._recommenders[position]
    def forgetting_list(self):
        return self.forgettings

In [3]:
import numpy as np

class Recall:
    def __init__(self, n):
        self.n = n
        self.hits = 0
        self.stream = 0 
        self.r_mean = 0.0
        self.recall_list = []
    def update(self, rank):
        if rank <= self.n:
            self.hits += 1
        self.stream += 1
        recall = self.score()
        self.recall_list.append(recall)
        return recall
    def score(self):
        return self.hits/self.stream
    def mean(self):
        return np.mean(self.recall_list)

In [4]:
experimenter = Experimenter()
recommender_list = []
recommender_list.append(MFRecommender)

# [1.01, 1.1, 10, 100, 1000, 10000]
# mean_pop = [-0.00040199748722443427, -0.0003777209117588382, -0.0003762619662848127, 
#             -0.0003814282091102922, -0.00038355540091335725, -0.0003867758902406115]
# alpha = np.array([.9999, .99, .95, .9, .8])
# alpha = alpha ** (1./np.mean(mean_pop))
# print("Alphas: {}".format(alpha))

experimenter.append(recommender_list, [NoForgetting], [None])
experimenter.append(recommender_list, [SDUserFactorFading], [1.08, 10, 100, 1000, 10000])
experimenter.append(recommender_list, [ForgetUnpopularItems], [1.2, 10, 100, 1000, 10000])
experimenter.append(recommender_list, [UserFactorFading], [.99, .95, .9, .85, .80])
experimenter.append(recommender_list, [MappedUserFactorFading], [1.1, 10, 100, 1000, 10000])

In [5]:
experiments = {}
for f in experimenter.forgetting_list():
    experiments[f] = {}
    experiments[f]['mean_recall'] = 0.0

for rec in experimenter:
    forgetting = rec.forgetting.__class__.__name__
    logging.info('initialize recommendation model {} with {}'.format(rec.__class__.__name__, forgetting))
    rec.initialize()
    evaluator = Evaluator(rec, data.can_repeat)

    n_batch_train = int(data.n_sample * 0.2)  # 20% for pre-training to avoid cold-start
    n_batch_test = int(data.n_sample * 0.3)  # 30% for evaluation of pre-training
    batch_tail = n_batch_train + n_batch_test

    # pre-train
    # 20% for batch training 
    batch_training = data.samples[:n_batch_train] 
    # 30% for batch evaluate
    batch_test = data.samples[n_batch_train:batch_tail]
    # after the batch training, 30% samples are used for incremental updating
    
    logging.info('batch pre-training before streaming input')
    evaluator.fit(
        batch_training,
        batch_test,
        max_n_epoch=20 
    )
    
    recall = Recall(RECALL_AT)
    
    recommendations = evaluator.recommend(batch_test)
    logging.info('measuring the recall@{} from the new model'.format(RECALL_AT))
    # (top-1 score, rank, recommend_time)
    for _, rank, _ in recommendations:
        recall.update(rank)
    mean = recall.mean() 
    logging.info('mean recall@{}: {}'.format(RECALL_AT,mean))
    if mean > experiments[forgetting]['mean_recall']:
        experiments[forgetting]['mean_recall'] = mean
        experiments[forgetting]['evaluator'] = evaluator.evaluate(data.samples[batch_tail:])
        experiments[forgetting]['recommender'] = rec.__class__.__name__
        experiments[forgetting]['alpha'] = rec.forgetting.alpha
        

# 50% incremental evaluation and updating
logging.info('incrementally predict, evaluate and update the recommender')    
for forgetting in experiments:
    logging.info('Recommender: {0}, Forgetting: {1}, Alpha: {2}, Mean Recall: {3:2f}'.format(
        experiments[forgetting]['recommender'],
        forgetting,
        experiments[forgetting]['alpha'],
        experiments[forgetting]['mean_recall']
    ))

2019-03-25 23:19:43,044 : INFO : initialize recommendation model MFRecommender with NoForgetting
2019-03-25 23:19:43,047 : INFO : batch pre-training before streaming input
2019-03-25 23:21:48,012 : INFO : 12888 : Epochs:14 Convergence:-6.412353865448139e-06
2019-03-25 23:21:48,012 : INFO : Epochs:14 Convergence:-6.412353865448139e-06
2019-03-25 23:21:51,694 : INFO : measuring the recall@10 from the new model
2019-03-25 23:24:12,587 : INFO : mean recall@10: 0.001305800926020611
2019-03-25 23:24:12,589 : INFO : initialize recommendation model MFRecommender with SDUserFactorFading
2019-03-25 23:24:12,591 : INFO : batch pre-training before streaming input
2019-03-25 23:26:40,822 : INFO : 12888 : Epochs:16 Convergence:-1.1652452336363694e-08
2019-03-25 23:26:40,822 : INFO : Epochs:16 Convergence:-1.1652452336363694e-08


Alpha:1.08 Mean:0.999972


2019-03-25 23:26:49,688 : INFO : measuring the recall@10 from the new model
2019-03-25 23:29:09,885 : INFO : mean recall@10: 0.0008883727843374939
2019-03-25 23:29:09,887 : INFO : initialize recommendation model MFRecommender with SDUserFactorFading
2019-03-25 23:29:09,889 : INFO : batch pre-training before streaming input
2019-03-25 23:31:08,344 : INFO : 12888 : Epochs:13 Convergence:-8.511639481789679e-06
2019-03-25 23:31:08,344 : INFO : Epochs:13 Convergence:-8.511639481789679e-06


Alpha:10 Mean:0.999129


2019-03-25 23:31:17,061 : INFO : measuring the recall@10 from the new model
2019-03-25 23:33:34,307 : INFO : mean recall@10: 0.0018307844362052694
2019-03-25 23:33:34,310 : INFO : initialize recommendation model MFRecommender with SDUserFactorFading
2019-03-25 23:33:34,313 : INFO : batch pre-training before streaming input
2019-03-25 23:35:52,771 : INFO : 12888 : Epochs:15 Convergence:-1.0732825250769906e-06
2019-03-25 23:35:52,771 : INFO : Epochs:15 Convergence:-1.0732825250769906e-06


...

2019-03-26 00:51:49,955 : INFO : measuring the recall@10 from the new model
2019-03-26 00:54:15,355 : INFO : mean recall@10: 0.005444343465084473
2019-03-26 00:54:15,358 : INFO : initialize recommendation model MFRecommender with MappedUserFactorFading
2019-03-26 00:54:15,360 : INFO : batch pre-training before streaming input
2019-03-26 00:57:37,834 : INFO : 12888 : Epochs:21 Convergence:-5.752785584467457e-05
2019-03-26 00:57:37,834 : INFO : Epochs:21 Convergence:-5.752785584467457e-05


Alpha:1000 Mean:0.847992


2019-03-26 00:57:48,473 : INFO : measuring the recall@10 from the new model
2019-03-26 01:00:12,081 : INFO : mean recall@10: 0.00532615274420061
2019-03-26 01:00:12,082 : INFO : initialize recommendation model MFRecommender with MappedUserFactorFading
2019-03-26 01:00:12,096 : INFO : batch pre-training before streaming input
2019-03-26 01:03:32,843 : INFO : 12888 : Epochs:21 Convergence:-5.732356615140155e-05
2019-03-26 01:03:32,843 : INFO : Epochs:21 Convergence:-5.732356615140155e-05


Alpha:10000 Mean:0.844217


2019-03-26 01:03:43,273 : INFO : measuring the recall@10 from the new model
2019-03-26 01:06:05,995 : INFO : mean recall@10: 0.005110515056788937
2019-03-26 01:06:06,008 : INFO : incrementally predict, evaluate and update the recommender
2019-03-26 01:06:06,008 : INFO : Recommender: MFRecommender, Forgetting: NoForgetting, Alpha: None, Mean Recall: 0.001306
2019-03-26 01:06:06,009 : INFO : Recommender: MFRecommender, Forgetting: SDUserFactorFading, Alpha: 10000, Mean Recall: 0.002903
2019-03-26 01:06:06,009 : INFO : Recommender: MFRecommender, Forgetting: ForgetUnpopularItems, Alpha: 100, Mean Recall: 0.001330
2019-03-26 01:06:06,009 : INFO : Recommender: MFRecommender, Forgetting: UserFactorFading, Alpha: 0.8, Mean Recall: 0.005833
2019-03-26 01:06:06,010 : INFO : Recommender: MFRecommender, Forgetting: MappedUserFactorFading, Alpha: 100, Mean Recall: 0.005444


In [6]:
for forgetting in experiments:
    evaluator = experiments[forgetting]['evaluator']
    recommender = experiments[forgetting]['recommender']
    param = str(experiments[forgetting]['alpha'])
    filename = "D:\\recsys\\flurs\\results\\{}_{}_{}_{}.dat".format(recommender, forgetting, param, EXP_NAME)
    logging.info("Abrindo arquivo {} ...".format(filename))
    with open(filename, 'w+') as f:
        logging.info("Começando a gerar resultados ...")
        for instance in evaluator:
            f.write(str(instance))
    logging.info("Arquivo {} completo.".format(filename))
    

2019-03-26 01:06:06,018 : INFO : Abrindo arquivo D:\recsys\flurs\results\MFRecommender_NoForgetting_None_M2.dat ...
2019-03-26 01:06:06,027 : INFO : Começando a gerar resultados ...
2019-03-26 01:11:26,565 : INFO : Arquivo D:\recsys\flurs\results\MFRecommender_NoForgetting_None_M2.dat completo.
2019-03-26 01:11:26,567 : INFO : Abrindo arquivo D:\recsys\flurs\results\MFRecommender_SDUserFactorFading_10000_M2.dat ...
2019-03-26 01:11:26,568 : INFO : Começando a gerar resultados ...
2019-03-26 01:16:53,842 : INFO : Arquivo D:\recsys\flurs\results\MFRecommender_SDUserFactorFading_10000_M2.dat completo.
2019-03-26 01:16:53,843 : INFO : Abrindo arquivo D:\recsys\flurs\results\MFRecommender_ForgetUnpopularItems_100_M2.dat ...
2019-03-26 01:16:53,844 : INFO : Começando a gerar resultados ...
2019-03-26 01:22:19,007 : INFO : Arquivo D:\recsys\flurs\results\MFRecommender_ForgetUnpopularItems_100_M2.dat completo.
2019-03-26 01:22:19,008 : INFO : Abrindo arquivo D:\recsys\flurs\results\MFRecommend